링크 : [Neural Granger Causality](https://arxiv.org/abs/1802.05842)

### 2. Linear Granger causality

`-` Granger causality : 한 시계열의 과거 값이 다른 시계열이 미래 값을 예측하는데 도움이 되는지를 평가하는 개념

`-` VAR(Vector Autogressive Regression)모델
* Granger causality를 분석하는 가장 기본적인 방법
* 시계열 데이터 표현 $x_t=\sum_{k=1}^{K}A^{(k)}x_{t-k}+\epsilon_t$
* $x_{t}$ : 시점 t에서의 다변량 시계열 데이터(p개의 변수 포함)
* k : 최대 시차
* $A^{(k)}$ : 시차 k에서의 계수행렬(pxp크기)
* $e_t$ : 평균이 0인 잡음

`-` 인과성 판단 기준 - 특정 시계열 j가 다른 시계열 i의 미래를 예측하는데 기여하는지를 확인
* $A^k_{ij}=0$이면, 시계열 j는 시계열 i에 대해 Granger 비인과적
* $A^k_{ij}\neq 0$이면, 시계열 j는 시계열 i에 대해 Granger 인과적

`-` 희소성 유도 패널티 적용
* 너무 많은 변수를 고려하면 과적합문제 발생->Lasso,Group Lasso 패널티를 적용하여 불필요한 계수 0으로
    * Group Lasso 패널티 : 결과적으로 희소한 Granger 인과 네트워크를 학습
        * $min_{A(1),...,A(K)}\sum_{t=K}^{T}||x_t-\sum_{k=1}^{K}A^{(k)}x_{t-k}||^2_2+\lambda\sum_{ij}||(A^{(1)}_{ij},...,A^{K}_{ij}||_2,$
    * 계층적 패널티 : 너무 긴 시차를 사용하면 과적합이 발생할 수 있으므로 모델이 적절한 시차로 학습해아함

`-` VAR은 선형적 관계에서만 가정, 비선형 데이터는 한계
* $\to$ Neural Granger Causality 기법 필요

### 3. Models for Neural Granger Causality

#### 3.1 Adapting Neural Networks for Granger Causality
`-` 비선형 자기 회귀 모델(Nonlinear Autogressive Model, NAR)
* $x_t=g(x_{<t1},...,x_{<tp})+\epsilon_t$
* $x_t$ : 시점 t에서의 다변량 시계열 데이터
* $x_{<t1},...,x_{<tp}$ : 과거 p개의 시점에서 수집된 데이터
* $g(*)$ : 비선형 함수(ex: 신경망)
* $e_t : 평균이 0인 잡음(noise)$

`-` Problem
* 블랙박스 문제 : MLP/LSTM은 강력한 예측성능을 가지지만 학습된 모델이 어떤 변수의 영향을 받는지 해석하기 어려움
* 시간지연(Lag) 선택문제 : 전통적인 VAR모델에서는 최대 시차 k를 설정해야 하는데, 신경망 모델에서는 이를 자동으로 결정하는 방법이 필요

`-` 해결방법
* cMLP,cLSTM : Component-wise Neural Network
* 아이디어 : 각 출력 변수 $x_{ti}$마다 독립적인 신경망 $g_i$를 사용하여 입력변수와의 관계를 학습
* 기존과 다르게 입력 가중치에 대한 희소성 패널티를 추가해 Granger Causality 분석가능
* 예시 : $x_{ti}=g_i(x_{<t1},...,x_{<tp})+e_t$
* $g_i(*)$ 변수 $i$에 대한 신경망 모델, 특정변수 $j$가 $i$에 영향을 미치지 않는다면, $g_{-i}$에서 $x_{<tj}$에 대해 불변해야함

`-` 3가지 모델 소개
* cLSTM
* cMLP
* cRNN